In [1]:
from data.dataset import Synth90kDataset, synth90k_collate_fn
import torch.multiprocessing as mp
import torch
from torch.utils.data import DataLoader

# Set multiprocessing start method to 'spawn'
mp.set_start_method('spawn', force=True)


dataset_path = './data/mnt/ramdisk/max/90kDICT32px/'
modes = ['train', 'val', 'test']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
test_dataset = Synth90kDataset(dataset_path, modes[2])

Loading Dataset with mode: test


In [3]:
test_dataset[2]

./data/mnt/ramdisk/max/90kDICT32px/./3000/7/180_Chronographs_13538.jpg


{'image': tensor([[[-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5686],
          [-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5608],
          [-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5608],
          ...,
          [-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5608],
          [-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5608],
          [-0.5608, -0.5608, -0.5608,  ..., -0.5608, -0.5608, -0.5608]]]),
 'target': tensor([12, 17, 27, 24, 23, 24, 16, 27, 10, 25, 17, 28]),
 'target_length': tensor([12, 17, 27, 24, 23, 24, 16, 27, 10, 25, 17, 28])}

In [4]:




test_loader = DataLoader(test_dataset, batch_size = 64,
                         shuffle = True, num_workers = 2,
                        collate_fn = synth90k_collate_fn)